In [4]:
from langchain.prompts import PromptTemplate


match_prompt_auto_fewshot_1 = PromptTemplate(
    input_variables=["p1", "p2"],
    template="""Q: Could you please tell me if the following two products are the same one?
    Product 1:
    Title: Kingston ValueRAM 4GB - PC3-12800 - DIMM
Description: ValueRam
    Product 2:
    Title: TARJETA DE RED PCI EXPRESS X1 INALAMBRICA TP-LINK WIRELESS 802.11N/G/B150MBPS ANTENA DESMONTA 2DBI
Description: nan
A: Let's think step by step! Product 1 is a Kingston ValueRAM 4GB memory module with a PC3-12800 specification and DIMM form factor. It is a computer component used to increase the system's memory capacity. On the other hand, Product 2 is a TP-Link wireless network card with a PCI Express x1 interface and 802.11n/g/b wireless standards. It is used to connect a computer to a wireless network. Therefore, these two products are not the same, and they serve different purposes.
Q: Could you please tell me if the following two products are the same one?
    Product 1:
    Title: Xerox 106R01162 25k Yellow Toner
Description: FeaturesPage yield25000 pagesBrand compatibilityXeroxCompatible productsPhaser 7760Colour of productBlackQuantity1Printing coloursYellowTypeTonerPackaging dataPackaging width95 mmPackaging height89 mmPackage weight680 gPackaging length368 mmPackaging
    Product 2:
    Title: 106R01162 Toner yellow pentru Phaser 7760
Description: yellow, 25000p for Phaser 7760
A: Let's think step by step! First, let's compare the titles of the two products. They both mention "106R01162" and "yellow toner" for the Phaser 7760 printer. So, it seems like they could be the same product.

Next, let's compare the descriptions. Both mention a page yield of 25000 pages and that the toner is yellow. The first product has more detailed information about the brand compatibility, packaging dimensions, and weight. The second product is more concise and only mentions the page yield and printer compatibility.

Based on this comparison, it seems likely that the two products are the same one. However, it's always a good idea to double-check with the seller or manufacturer to confirm.
Q: Could you please tell me if the following two products are the same one?
    Product 1:
    {p1}
    Product 2:
    {p2}
A: Let's think step by step!
"""
)

In [5]:
auto_cot_prompt_1 = PromptTemplate(
    input_variables=["p1", "p2"],
    template="""Q: Could you please tell me if the following two products are the same one?
    Product 1:
    {p1}
    Product 2:
    {p2}
A: Let's think step by step! """
)

auto_cot_prompt_2 = PromptTemplate(
    input_variables=["p1", "p2"],
    template="""Q: Are the following two products the same?
Product 1: 
{p1}
Product 2: 
{p2}

A: Let's think step by step! """
)



In [6]:
eval_prompt_2 = PromptTemplate(
    input_variables=["chain_of_thought_reasoning"],
    template="""Decide if this answer corresponds to two products matching or not.
    Please answer with "match" or "no match".
    Answer: {chain_of_thought_reasoning}
"""
)

In [7]:
def cut_length(string ,length = 250):
    return string[0:length]

In [8]:
import pandas as pd
from langchain.chains import LLMChain, SequentialChain
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()


True

In [9]:
from langfuse.callback import CallbackHandler
handler_auto_cot = CallbackHandler(os.environ.get("LF_PK"), os.environ.get("LF_SK"), os.environ.get("LF_HOST"))
handler = CallbackHandler(os.environ.get("LF_PK"), os.environ.get("LF_SK"), os.environ.get("LF_HOST"))

In [10]:
df_train = pd.read_csv("../datasets/training_data_50_50.csv")
df_amazon_walmart_no_match = pd.read_csv("../datasets/amazon_walmart_NDPL.tsv", sep="\t")
df_amazon_walmart_match = pd.read_csv("../datasets/amazon_walmart_DPL.tsv", sep="\t")
df_products = pd.read_csv("../datasets/amazon_walmart.tsv", sep="\t")
save_set_amazon_walmart = pd.read_csv("../datasets/amazon_walmart_auto_cot_reasoning.csv") 

# CoT-gpt3.5turbo-0301
# eval-gpt3.5turbo-0301
# correct-gpt3.5turbo-0301


In [11]:
def simple_auto_cot_prompt(dataset, auto_cot_prompt, model="gpt-3.5-turbo-0301", start=0, end=10):
    if dataset == "50_50":
        save_set = df_train
    else:
        save_set = save_set_amazon_walmart
    for i in range(start, end+1):
        print(i)
        reasoning = ""
        llm = ChatOpenAI(model_name= model, temperature=0)
        matching = False
        if dataset == "50_50":
            data = df_train
            product1 = "Title: " + cut_length(str(data["title_left"][i] or "none")) + "\nDescription: " + cut_length(str(data["description_left"][i] or "none")) 
            product2 = "Title: " + cut_length(str(data["title_right"][i] or "none")) + "\nDescription: " + cut_length(str(data["description_right"][i] or "none")) 
        elif dataset == "amazon_walmart":
            if (i%2==0):
                data = df_amazon_walmart_match
                matching = True
            else:
                data = df_amazon_walmart_no_match
            product1 = "Title: " + cut_length(str(df_products["title"][data["id1"][i]-1] or "none")) + "\nDescription: " + cut_length(str(df_products["longdescr"][data["id1"][i]-1] or "none")) 
            product2 = "Title: " + cut_length(str(df_products["title"][data["id2"][i]-1] or "none")) + "\nDescription: " + cut_length(str(df_products["longdescr"][data["id2"][i]-1] or "none")) 
            save_set.loc[i, "id1"] = data["id1"][i]
            save_set.loc[i, "id2"] = data["id2"][i]
            save_set.loc[i, "label"] = str(matching)
        else:
            print("dataset not found")
            return

        cot_chain = LLMChain(
            llm=llm, prompt=auto_cot_prompt, callbacks=[handler_auto_cot]
        )
        reasoning += auto_cot_prompt.format_prompt(p1=product1, p2=product2).text
        try:
            answer = cot_chain.run(
                p1 = product1,
                p2 = product2,
                callbacks=[handler_auto_cot]
            )      
        except Exception as e:
            print(e)
            i -= 1
            continue
        reasoning += answer
        reasoning += "\n"

        # save reasoning to df_train in column "CoT-gpt3.5turbo-0301"
        save_set.loc[i, "CoT-"+model] = str(reasoning)
        print(str(reasoning))

        eval_chain = LLMChain(
            llm=llm, prompt=eval_prompt_2, callbacks=[handler]
        )
        try:
            eval_output = eval_chain.run(
                chain_of_thought_reasoning = answer,
                callbacks=[handler]
            )
        except Exception as e:
            print(e)
            i -= 1
            continue
        
        is_match = False
        if  "No" in eval_output or "no" in eval_output:
            is_match = False
        elif ("Match" in eval_output) or ("match" in eval_output):
            is_match = True
        else:
            print(eval_output)
            break

        save_set.loc[i, "eval-"+model] = str(is_match)
        if(dataset == "50_50"):
            save_set.loc[i, "correct-"+model] = str(is_match == save_set["label"][i])
        if (dataset == "amazon_walmart"):
            save_set.loc[i, "correct-"+model] = str(is_match == matching)
    if dataset== "50_50":
        save_set.to_csv("../datasets/training_data_50_50.csv", index=True)
    elif dataset == "amazon_walmart":
        save_set.to_csv("../datasets/amazon_walmart_auto_cot_reasoning.csv", index=True)



In [14]:
simple_auto_cot_prompt("50_50", auto_cot_prompt_2, model="gpt-4-0613",  start=68, end=69)

68
Q: Are the following two products the same?
Product 1: 
Title: Daniel Wellington Classic St Mawes 40mm
Description: The Classic St. Mawes, an integral part of the flagship Classic Collection, is a slim timepiece that sits perfectly on your wrist. With a flawlessly round and simple dial, a classy leather band and an elegant casing, you have a timepiece that proves 
Product 2: 
Title: St Mawes 40mm Rose Gold Watch - Multi
Description: An integral part of the flagship classic Daniel Wellington collection, featuring a classy brown leather band a stunning rose gold case, perfectly engineered with a flawlessly round and simple dial.This watch sits perfectly on your wrist and has the a

A: Let's think step by step! Both products seem to be watches from the Daniel Wellington Classic Collection, specifically the St. Mawes 40mm model. They both have a round and simple dial, a leather band, and sit perfectly on the wrist. However, Product 2 specifically mentions a rose gold case, while Product

In [ ]:
# save df_train to csv
# df_train.to_csv("../datasets/training_data_50_50.csv", index=False)

In [28]:
def createAutoCotPrompt(dataset, n=2, model="gpt3.5turbo-0301"):
    if dataset == "50_50":
        fewShotData = pd.read_csv("../datasets/training_data_50_50.csv")
    elif dataset == "amazon_walmart":
        fewShotData = pd.read_csv("../datasets/amazon_walmart_auto_cot_reasoning.csv")
    else:
        print("dataset not found")
        return
    fewShotData = fewShotData[fewShotData["correct-"+ model] == True]
    fewShotData = fewShotData.sample(n=n)
    template = ""
    for _, row in fewShotData.iterrows():
        template += row["CoT-"+ model] + "\n"
        template += "The two products " + ("match" if row["label"] == True else "do not match") + ".\n"
    template = "\n".join([s for s in template.splitlines() if s.strip()])
    template += """\nQ: Could you please tell me if the following two products are the same one?
Product 1:
{p1}
Product 2:
{p2}
A: Let's think step by step!
"""
    print(template)

    prompt = PromptTemplate(
        input_variables=["p1", "p2"],
        template=template
    )

    return prompt





In [37]:
print(createAutoCotPrompt(dataset="amazon_walmart", model="gpt-3.5-turbo-0301"))

Q: Are the following two products the same?
Product 1: 
Title: hp laserjet pro p1606dn printer ce749a bgj
Description: why buy save paper and time with auto two sided printing hp auto on auto off technology 1 easy printer setup without using a cd 2 enlarge functions features enlarge built in wireless printing print from any room without using cables built in wired ne
Product 2: 
Title: canon cli 8y yellow inkjet print cartridge
Description: canon cli 8y yellow inkjet print cartridge print technology inkjet color yellow model cli 8y compatibility pixma ip3300 pixma ip3500 pixma ip4200 pixma ip4200 refurbished pixma ip4300 pixma ip4500 pixma ip5200 pixma ip5200r pixma ip6600d pixma ip6600
A: Let's think step by step! First, we need to identify what the products are. 
Product 1 is a printer, specifically the HP LaserJet Pro P1606dn Printer. 
Product 2 is an inkjet print cartridge, specifically the Canon CLI 8Y Yellow Inkjet Print Cartridge. 
So, no, the two products are not the same. They

In [ ]:
# def simple_auto_cot_prompt(auto_cot_prompt, indexes=[], n=1):
#     if len(indexes) > 0:
#         n = len(indexes)
#     template = ""
#     for j in range(n):
#         if len(indexes) > 0:
#             i = indexes[j]
#         else:
#             i = i
#         llm = ChatOpenAI(model_name= "gpt-3.5-turbo-0301", temperature=0)

#         product1 = "Title: " + cut_length(str(df_train["title_left"][i] or "none")) + "\nDescription: " + cut_length(str(df_train["description_left"][i] or "none")) 
#         product2 = "Title: " + cut_length(str(df_train["title_right"][i] or "none")) + "\nDescription: " + cut_length(str(df_train["description_right"][i] or "none")) 
        

#         chain = LLMChain(
#             llm=llm, prompt=auto_cot_prompt, callbacks=[handler_auto_cot]
#         )
#         template += auto_cot_prompt.format_prompt(p1=product1, p2=product2).text
#         try:
#             answer = chain.run(
#                 p1 = product1,
#                 p2 = product2,
#                 callbacks=[handler_auto_cot]
#             )      
#         except Exception as e:
#             print(e)
#             i -= 1
#             continue
#         template += answer
#         template += "\n"
#     template += auto_cot_prompt.template
#     overall_prompt = PromptTemplate(
#         input_variables=["p1", "p2"],
#         template=template
#     )
#     # print(overall_prompt.template)
#     return overall_prompt




In [ ]:
# print(simple_auto_cot_prompt(auto_cot_prompt_1, [4,6], 2).template)